In [1]:
import pandas as pd

finetuned_dirname = "40-epoch-roberta-finetuned-phemernr2-rnr-check-finetune"

data = pd.read_csv("../../data/phemernr2_dataset_with_tvt.csv", sep=",")
# data = data[['tweet_text', 'tvt2', 'label2']]
# data.columns = ['tweet_text', 'tvt2', 'label']
print(data.shape)
data.head()

(6425, 5)


,tweet_id,tweet_text,label,tvt2,tvt2_1
0,552833795142209536,the east london mosque would like to offer its...,non-rumours,training,training
1,580318210609696769,breaking - a germanwings airbus a320 plane rep...,true,validation,testting
2,552798891994009601,reports that two of the dead in the #charliehe...,true,training,training
3,576790814942236672,after #putin disappeared russian tv no longer ...,non-rumours,validation,training
4,499678822598340608,saw #ferguson for myself. #justiceformichaelbr...,non-rumours,testting,testting


In [2]:
data['label'] = data['label'].replace(['true', 'unverfied', 'false'], 'rumors')

In [3]:
combined_data = data

In [4]:
import torch

class CustomTextDataset(torch.utils.data.dataset.Dataset):

    def __init__(self, texts, labels):
        self.labels = labels
        self.texts = texts
        self.attention_mask = None
        self.input_ids = None
        self.token_type_ids = None

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        sample = {
            "text": self.texts[idx],
            "label": self.labels[idx],
            "attention_mask": self.attention_mask[idx] if self.attention_mask else None,
            "input_ids": self.input_ids[idx] if self.input_ids else None,
        }
        return sample
    
    def tokenize(self, tokenizer):
        self.attention_mask = []
        self.input_ids = []
        self.token_type_ids = []

        for text in self.texts:
            token = tokenizer(text, padding="max_length", truncation=True)
            
            self.attention_mask.append(token['attention_mask'])
            self.input_ids.append(token['input_ids'])

c:\Users\msi_b\anaconda3\envs\pytorch-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
labels = []

labels_str = combined_data['label'].unique().tolist()
for i, d in combined_data.iterrows():
    if d['label'] == "non-rumours":
        lab = 1
    else:
        lab = 0
    labels.append(lab)
    
print(len(labels))
labels[:10]

6425


[1, 0, 0, 1, 1, 1, 1, 0, 1, 1]

In [6]:
train_dataset = CustomTextDataset(
    [d['tweet_text'] for i, d in combined_data.iterrows() if d['tvt2'] == 'training'],
    [labels[i] for i, d in combined_data.iterrows() if d['tvt2'] == 'training'])
test_dataset = CustomTextDataset(
    [d['tweet_text'] for i, d in combined_data.iterrows() if d['tvt2'] == 'validation'],
    [labels[i] for i, d in combined_data.iterrows() if d['tvt2'] == 'validation'])
train_dataset[0]

{'text': 'the east london mosque would like to offer its sincere condolences to the families of those killed during the #charliehebdo attacks (1/2)',
 'label': 1,
 'attention_mask': None,
 'input_ids': None}

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base")

In [8]:
# inputs = tokenizer(["you're stuck in a timewrap from 2004 though", "summa lumma dumma lumma"], padding="max_length", truncation=True)
# for k,v in inputs.items():
#     print(k)

In [9]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

train_dataset.tokenize(tokenizer)
test_dataset.tokenize(tokenizer)

In [10]:
print(len(train_dataset))
print(len(test_dataset))

4336
1462


### Fine Tuning

In [11]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("roberta-base",
                                                           output_hidden_states=False,
                                                           num_labels=2)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
from transformers import TrainingArguments

epochs = 40
batch_size = 8
save_steps = (round((len(train_dataset)/batch_size) + 0.49)) * epochs
# save_steps = 1_000_000

training_args = TrainingArguments(
    output_dir=f"../../data/models/{finetuned_dirname}",
    num_train_epochs=epochs,
    save_steps=save_steps,
    logging_steps=300,
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    evaluation_strategy="epoch",
    logging_strategy="epoch"
)

print(f"Save Steps : {save_steps}")

Save Steps : 21680


In [13]:
import numpy as np
from datasets import load_metric
# from copy import deepcopy

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# def compute_metrics(eval_pred):
#     metrics = ["accuracy", "recall", "precision", "f1"] #List of metrics to return
#     metric={}
#     for met in metrics:
#        metric[met] = load_metric(met)
#     logits, labels = eval_pred
#     predictions = np.argmax(logits, axis=-1)
#     metric_res={}
#     for met in metrics:
#        metric_res[met]=metric[met].compute(predictions=predictions, references=labels)[met]
#     return metric_res

C:\Users\msi_b\AppData\Local\Temp\ipykernel_21456\3274651076.py:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [14]:
# class CustomCallback(TrainerCallback):
    
#     def __init__(self, trainer) -> None:
#         super().__init__()
#         self._trainer = trainer
    
#     def on_epoch_end(self, args, state, control, **kwargs):
#         if control.should_evaluate:
#             control_copy = deepcopy(control)
#             self._trainer.evaluate(eval_dataset=self._trainer.train_dataset, metric_key_prefix="train")
#             return control_copy

In [15]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

In [16]:
import time

start = time.time()

train = trainer.train()

print(f"Execution Time : {round(time.time() - start)} seconds")

  2%|▎         | 542/21680 [02:47<1:48:54,  3.24it/s]

{'loss': 0.4826, 'learning_rate': 9.75e-06, 'epoch': 1.0}


                                                     
  2%|▎         | 542/21680 [03:03<1:48:54,  3.24it/s]

{'eval_loss': 0.4852116107940674, 'eval_accuracy': 0.8160054719562243, 'eval_runtime': 16.718, 'eval_samples_per_second': 87.45, 'eval_steps_per_second': 10.946, 'epoch': 1.0}


  5%|▌         | 1084/21680 [05:49<1:45:09,  3.26it/s]

...

{'eval_loss': 1.1774280071258545, 'eval_accuracy': 0.8926128590971272, 'eval_runtime': 16.7644, 'eval_samples_per_second': 87.209, 'eval_steps_per_second': 10.916, 'epoch': 39.0}


100%|██████████| 21680/21680 [2:01:40<00:00,  3.25it/s]

{'loss': 0.0027, 'learning_rate': 0.0, 'epoch': 40.0}


                                                       
100%|██████████| 21680/21680 [2:01:57<00:00,  2.96it/s]

{'eval_loss': 1.1771156787872314, 'eval_accuracy': 0.893296853625171, 'eval_runtime': 16.7531, 'eval_samples_per_second': 87.267, 'eval_steps_per_second': 10.923, 'epoch': 40.0}
{'train_runtime': 7317.7142, 'train_samples_per_second': 23.701, 'train_steps_per_second': 2.963, 'train_loss': 0.0608820932336509, 'epoch': 40.0}
Execution Time : 7318 seconds


In [17]:
trainer.evaluate()

100%|██████████| 183/183 [00:16<00:00, 11.01it/s]


{'eval_loss': 1.1771156787872314,
 'eval_accuracy': 0.893296853625171,
 'eval_runtime': 16.7214,
 'eval_samples_per_second': 87.433,
 'eval_steps_per_second': 10.944,
 'epoch': 40.0}